In [1]:
from nml_rst_proc import gen_url_option, set_input, set_site, set_initcond, set_runcontrol, set_input_initcond, set_input_runcontrol, parse_option_rst
from urlpath import URL
import urllib.request
import numpy as np
import pandas as pd
import supy as sp
import os
os.getcwd()


'/Users/sunt05/Dropbox/8-Research/98.ReadingWork/SuPy/docs/proc_var_info'

## generate site characteristics related dataframe

### load `SUEWS_***.txt` related tables

In [2]:
list_table = [file.replace('.txt', '.csv')
              for file in sp.supy_load.list_file_input
              if 'Profile' not in file]  # exclude `Profile` txt: a special case

url_repo_base = 'https://github.com/Urban-Meteorology-Reading/SUEWS-Docs/raw/master/docs/source'
url_table_base = (
    url_repo_base
    + '/input_files/SUEWS_SiteInfo/csv-table/'
)

# url_table_base = (
#     '/Users/sunt05/Dropbox/8-Research/98.ReadingWork/SUWES-Docs/'
#     + 'docs/source/input_files/SUEWS_SiteInfo/csv-table/')

list_url_table = [url_table_base + table for table in list_table]


#### retrieve SUEWS variable descriptions

In [3]:
df_var_info = pd.concat([pd.read_csv(f) for f in list_url_table])
df_var_info_x = df_var_info.drop(['No.', 'Use'], axis=1)
df_var_info_x = df_var_info_x.set_index('Column Name')
df_var_info_x.index = df_var_info_x.index.map(lambda x: x.replace('`', ''))
df_var_info_x.head()


,Description
Column Name,
Code,Code linking to a corresponding look-up table.
BaseTHDD,Base temperature for heating degree days [°C]
QF_A_WD,Base value for QF on weekdays [W |m^-2| (Cap |...
QF_B_WD,Parameter related to heating degree days on we...
QF_C_WD,Parameter related to heating degree days on we...


#### retrieve SUEWS-related variable names

In [4]:
# retrieve SUEWS-related variables
dict_var2SiteSelect = sp.supy_load.dict_var2SiteSelect

dict_var_full = sp.supy_load.exp_dict_full(dict_var2SiteSelect)


def extract_var_suews(dict_var_full, k):
    x = sp.supy_load.flatten_list(dict_var_full[k])
    x = np.unique(x)
    x = [
        xx for xx in x
        if xx not in ['base', 'const', '0.0'] + [str(x) for x in range(24)]
    ]
    x = [xx for xx in x if 'Code' not in xx]
    return x


dict_var_ref_suews = {
    k: extract_var_suews(dict_var_full, k)
    for k in dict_var_full
}

df_var_ref_suews = pd.DataFrame(
    {k: ', '.join(dict_var_ref_suews[k])
     for k in dict_var_ref_suews},
    index=[0]).T.rename({
        0: 'SUEWS-related variables'
    }, axis=1)

# df_var_ref_suews.head(10)
# set_input_site_exp0=df_var_ref_suews.loc[:,'SUEWS-related variables'].str.lower().str.split(',')

# set_input_site_exp= set(x.lower() for x in np.concatenate(set_input_site_exp0.values))



In [5]:
# retrive supy variable description
dict_var_desc = {k: '\n'.join(df_var_info_x.loc[v].values.flatten())
                 for k, v in dict_var_ref_suews.items()}

df_var_desc = pd.DataFrame(dict_var_desc, index=[0]).T.rename({
    0: 'Description'}, axis=1)

df_var_desc.head()


,Description
ah_min,Minimum QF on weekdays [W |m^-2|]\nMinimum QF ...
ah_slope_cooling,Cooling slope of QF on weekdays [W |m^-2| |K^-...
ah_slope_heating,Heating slope of QF on weekdays [W |m^-2| |K^-...
ahprof_24hr,Code linking to `EnergyUseProfWD` in `SUEWS_Pr...
alb,Effective surface albedo (middle of the day va...


#### list supy variables that are combinations of multiple SUEWS variables

In [6]:
for k, v in dict_var_ref_suews.items():
    if len(v) > 1:
        print('')
        print(k, v)
    # df_var_info_x.loc[v].values.flatten()



ah_min ['AHMin_WD', 'AHMin_WE']

ah_slope_cooling ['AHSlope_Cooling_WD', 'AHSlope_Cooling_WE']

ah_slope_heating ['AHSlope_Heating_WD', 'AHSlope_Heating_WE']

ahprof_24hr ['EnergyUseProfWD', 'EnergyUseProfWE']

daywat ['DayWat(1)', 'DayWat(2)', 'DayWat(3)', 'DayWat(4)', 'DayWat(5)', 'DayWat(6)', 'DayWat(7)']

daywatper ['DayWatPer(1)', 'DayWatPer(2)', 'DayWatPer(3)', 'DayWatPer(4)', 'DayWatPer(5)', 'DayWatPer(6)', 'DayWatPer(7)']

humactivity_24hr ['ActivityProfWD', 'ActivityProfWE']

ie_a ['Ie_a1', 'Ie_a2', 'Ie_a3']

ie_m ['Ie_m1', 'Ie_m2', 'Ie_m3']

laipower ['LeafGrowthPower1', 'LeafGrowthPower2', 'LeafOffPower1', 'LeafOffPower2']

ohm_coef ['a1', 'a2', 'a3']

popprof_24hr ['PopProfWD', 'PopProfWE']

qf0_beu ['QF0_BEU_WD', 'QF0_BEU_WE']

qf_a ['QF_A_WD', 'QF_A_WE']

qf_b ['QF_B_WD', 'QF_B_WE']

qf_c ['QF_C_WD', 'QF_C_WE']

sfr ['Fr_Bldgs', 'Fr_Bsoil', 'Fr_DecTr', 'Fr_EveTr', 'Fr_Grass', 'Fr_Paved', 'Fr_Water']

snowprof_24hr ['SnowClearingProfWD', 'SnowClearingProfWE']

storedrainp

#### retrieve variable dimensionality

In [7]:
df_init_sample, df_forcing_sample = sp.load_SampleData()
df_var_dim_x = df_init_sample.columns.to_frame()
df_var_dim_x.index = df_var_dim_x.index.droplevel(-1)

df_var_dim_x = df_var_dim_x.groupby('var').last()

df_var_dim = df_var_dim_x.applymap(eval).applymap(
    lambda x: tuple(np.array(x) + 1) if isinstance(x, tuple) else x)

df_var_dim = df_var_dim.rename({'ind_dim': 'Dimensionality'}, axis=1)
df_var_dim = pd.DataFrame(df_var_dim)
df_var_dim.filter(like='method', axis=0)
df_var_dim.head()


,Dimensionality
var,
aerodynamicresistancemethod,0
ah_min,"(2,)"
ah_slope_cooling,"(2,)"
ah_slope_heating,"(2,)"
ahprof_24hr,"(24, 2)"


### select only those useful for input to `supy`

In [8]:

df_var_site_raw = pd.concat(
    [df_var_dim, df_var_desc, df_var_ref_suews], axis=1)
df_var_site = df_var_site_raw.filter(items=set_input, axis=0).dropna()
# df_var_site.to_csv('df_var_site.csv')



In [9]:
# input options other than those from site characteristics
set_input_site = set(df_var_site.index)
set_input_nonsite = set_input.difference(set_input_site)
set_input_nonsite


{'aerodynamicresistancemethod',
 'albdectr_id',
 'albevetr_id',
 'albgrass_id',
 'decidcap_id',
 'diagnose',
 'diagqn',
 'diagqs',
 'dqndt',
 'dqnsdt',
 'dt_since_start',
 'emissionsmethod',
 'evapmethod',
 'gdd_id',
 'gridiv',
 'hdd_id',
 'icefrac',
 'lai_id',
 'laicalcyes',
 'len_sim',
 'metforcingblock',
 'netradiationmethod',
 'numcapita',
 'ohmincqf',
 'porosity_id',
 'qn1_av',
 'qn1_s_av',
 'roughlenheatmethod',
 'roughlenmommethod',
 'smdmethod',
 'snowalb',
 'snowdens',
 'snowfallcum',
 'snowfrac',
 'snowpack',
 'snowuse',
 'snowwater',
 'soilstore_id',
 'stabilitymethod',
 'state_id',
 'storageheatmethod',
 'tair24hr',
 'tstep',
 'tstep_prev',
 'veg_type',
 'waterusemethod',
 'wuday_id'}

## process `runcontrol` and `initialcondition` related variables

### these variables are based on `nml` files so the processing logic is different

In [10]:
list_var_nml = list(set_initcond)+list(set_runcontrol)
list_url_rst_x = list(gen_url_option(var, source='github')
                      for var in sorted(list_var_nml))

# gen_url_option('tstep')
sorted(list_var_nml)



['aerodynamicresistancemethod',
 'albdectr0',
 'albevetr0',
 'albgrass0',
 'bldgsstate',
 'bsoilstate',
 'cbluse',
 'dayssincerain',
 'decidcap0',
 'dectrstate',
 'diagnose',
 'diagnosedisagg',
 'diagnosedisaggestm',
 'diagqn',
 'diagqs',
 'disaggmethod',
 'disaggmethodestm',
 'dqndt',
 'dqnsdt',
 'emissionsmethod',
 'evapmethod',
 'evetrstate',
 'filecode',
 'fileinputpath',
 'fileoutputpath',
 'gdd_1_0',
 'gdd_2_0',
 'grassstate',
 'kdownzen',
 'keeptstepfilesin',
 'keeptstepfilesout',
 'laicalcyes',
 'laiinitialdectr',
 'laiinitialevetr',
 'laiinitialgrass',
 'leavesoutinitially',
 'multipleestmfiles',
 'multipleinitfiles',
 'multiplemetfiles',
 'multrainamongn',
 'multrainamongnupperi',
 'ncmode',
 'ncol',
 'netradiationmethod',
 'nrow',
 'ohmincqf',
 'pavedstate',
 'porosity0',
 'qn1_av',
 'qn1_s_av',
 'rainamongn',
 'raindisaggmethod',
 'resolutionfilesin',
 'resolutionfilesinestm',
 'resolutionfilesout',
 'roughlenheatmethod',
 'roughlenmommethod',
 'smdmethod',
 'snowalb0',
 's

In [11]:
# URL('/'.join(list_var_url[0].parts)).get().ok
list_url_rst = list(set(URL('/'.join(url.parts)) for url in list_url_rst_x))



In [12]:

df_var_nml = pd.concat(parse_option_rst(url) for url in list_url_rst)
df_var_nml_x = df_var_nml.reset_index().loc[:, ['Description', 'index']].rename(
    columns={'index': 'SUEWS-related variables'})
index_nml = df_var_nml_x.loc[:, 'SUEWS-related variables'].str.lower()
df_var_nml_x.index = index_nml.rename('var')
df_var_nml_x.sort_index()


,Description,SUEWS-related variables
var,,
albdectr0,Albedo of deciduous surface `DecTr` on day 0 o...,albDecTr0
albevetr0,Albedo of evergreen surface `EveTr` on day 0 o...,albEveTr0
albgrass0,Albedo of grass surface `Grass` on day 0 of run,albGrass0
bldgsstate,Initial wetness condition on `Bldgs`,BldgsState
bsoilstate,Initial wetness condition on `BSoil`,BSoilState
cbluse,Determines whether a CBL slab model is used to...,CBLuse
dayssincerain,Days since rain [d],DaysSinceRain
decidcap0,Storage capacity of deciduous surface `DecTr` ...,decidCap0
dectrstate,Initial wetness condition on `DecTr`,DecTrState


### generate `df_var_runcontrol` for supy

In [13]:
df_var_runcontrol_x = df_var_nml_x.filter(items=set_input_runcontrol, axis=0)
df_var_runcontrol_dim = df_var_dim.filter(items=set_input_runcontrol, axis=0)
df_var_runcontrol = pd.concat(
    [df_var_runcontrol_dim, df_var_runcontrol_x], axis=1)
df_var_runcontrol


,Dimensionality,Description,SUEWS-related variables
aerodynamicresistancemethod,0,NaN,NaN
diagnose,0,NaN,NaN
diagqn,0,NaN,NaN
diagqs,0,NaN,NaN
emissionsmethod,0,Determines method for QF calculation.,EmissionsMethod
evapmethod,0,NaN,NaN
laicalcyes,0,NaN,NaN
netradiationmethod,0,Determines method for calculation of radiation...,NetRadiationMethod
ohmincqf,0,Determines whether the storage heat flux calcu...,OHMIncQF
roughlenheatmethod,0,Determines method for calculating roughness le...,RoughLenHeatMethod


### generate `df_var_initcond` for supy

#### define related variables for `initcond` variables

In [14]:
set_input_nonsite - set_input_runcontrol



{'albdectr_id',
 'albevetr_id',
 'albgrass_id',
 'decidcap_id',
 'dqndt',
 'dqnsdt',
 'dt_since_start',
 'gdd_id',
 'gridiv',
 'hdd_id',
 'icefrac',
 'lai_id',
 'len_sim',
 'metforcingblock',
 'numcapita',
 'porosity_id',
 'qn1_av',
 'qn1_s_av',
 'snowalb',
 'snowdens',
 'snowfallcum',
 'snowfrac',
 'snowpack',
 'snowwater',
 'soilstore_id',
 'state_id',
 'tair24hr',
 'tstep_prev',
 'wuday_id'}

In [15]:
set_initcond



{'albdectr0',
 'albevetr0',
 'albgrass0',
 'bldgsstate',
 'bsoilstate',
 'dayssincerain',
 'decidcap0',
 'dectrstate',
 'dqndt',
 'dqnsdt',
 'evetrstate',
 'gdd_1_0',
 'gdd_2_0',
 'grassstate',
 'laiinitialdectr',
 'laiinitialevetr',
 'laiinitialgrass',
 'leavesoutinitially',
 'pavedstate',
 'porosity0',
 'qn1_av',
 'qn1_s_av',
 'snowalb0',
 'snowdensbldgs',
 'snowdensbsoil',
 'snowdensdectr',
 'snowdensevetr',
 'snowdensgrass',
 'snowdenspaved',
 'snowdenswater',
 'snowfracbldgs',
 'snowfracbsoil',
 'snowfracdectr',
 'snowfracevetr',
 'snowfracgrass',
 'snowfracpaved',
 'snowfracwater',
 'snowinitially',
 'snowpackbldgs',
 'snowpackbsoil',
 'snowpackdectr',
 'snowpackevetr',
 'snowpackgrass',
 'snowpackpaved',
 'snowpackwater',
 'snowwaterbldgsstate',
 'snowwaterbsoilstate',
 'snowwaterdectrstate',
 'snowwaterevetrstate',
 'snowwatergrassstate',
 'snowwaterpavedstate',
 'snowwaterwaterstate',
 'soilstorebldgsstate',
 'soilstorebsoilstate',
 'soilstoredectrstate',
 'soilstoreevetrstate

In [16]:
dict_initcond_related = {
    'albdectr_id': 'albdectr0',
    'albevetr_id': 'albevetr0',
    'albgrass_id': 'albgrass0',
    'decidcap_id': 'decidcap0',
    # 'dqndt': 'dqndt',
    # 'dqnsdt': 'dqnsdt',
    # 'gdd_id': ['gdd_1_0', 'gdd_2_0', ],
    # 'hdd_id':,
    # 'icefrac':,
    'lai_id': [
        'laiinitialdectr',
        'laiinitialevetr',
        'laiinitialgrass',
    ],
    # 'numcapita':,
    'porosity_id': 'porosity0',
    # 'qn1_av': 'qn1_av',
    # 'qn1_s_av': 'qn1_s_av',
    'snowalb': 'snowalb0',
    'snowwater': [
        'snowwaterbldgsstate',
        'snowwaterpavedstate',
        'snowwaterdectrstate',
        'snowwaterevetrstate',
        'snowwatergrassstate',
        'snowwaterbsoilstate',
        'snowwaterwaterstate',
    ],
    'snowdens': [
        'snowdensbldgs',
        'snowdenspaved',
        'snowdensdectr',
        'snowdensevetr',
        'snowdensgrass',
        'snowdensbsoil',
        'snowdenswater',
    ],
    # 'snowfallcum':,
    'snowfrac': [
        'snowfracbldgs',
        'snowfracpaved',
        'snowfracdectr',
        'snowfracevetr',
        'snowfracgrass',
        'snowfracbsoil',
        'snowfracwater',
    ],
    'snowpack': [
        'snowpackbldgs',
        'snowpackpaved',
        'snowpackdectr',
        'snowpackevetr',
        'snowpackgrass',
        'snowpackbsoil',
        'snowpackwater',
    ],
    'soilstore_id': [
        'soilstorebldgsstate',
        'soilstorepavedstate',
        'soilstoredectrstate',
        'soilstoreevetrstate',
        'soilstoregrassstate',
        'soilstorebsoilstate',
        # 'soilstorewaterstate',
    ],
    'state_id': [
        'bldgsstate',
        'pavedstate',
        'dectrstate',
        'evetrstate',
        'grassstate',
        'bsoilstate',
        'waterstate',
    ],
    # 'tair24hr':,
    # 'tstep_prev':,
    # 'wuday_id':,
}


#### examine related variables

In [17]:
for var in dict_initcond_related:
    print(dict_initcond_related[var])
    df_var_nml_x.loc[dict_initcond_related[var], 'SUEWS-related variables']



albdectr0
albevetr0
albgrass0
decidcap0
['laiinitialdectr', 'laiinitialevetr', 'laiinitialgrass']
porosity0
snowalb0
['snowwaterbldgsstate', 'snowwaterpavedstate', 'snowwaterdectrstate', 'snowwaterevetrstate', 'snowwatergrassstate', 'snowwaterbsoilstate', 'snowwaterwaterstate']
['snowdensbldgs', 'snowdenspaved', 'snowdensdectr', 'snowdensevetr', 'snowdensgrass', 'snowdensbsoil', 'snowdenswater']
['snowfracbldgs', 'snowfracpaved', 'snowfracdectr', 'snowfracevetr', 'snowfracgrass', 'snowfracbsoil', 'snowfracwater']
['snowpackbldgs', 'snowpackpaved', 'snowpackdectr', 'snowpackevetr', 'snowpackgrass', 'snowpackbsoil', 'snowpackwater']
['soilstorebldgsstate', 'soilstorepavedstate', 'soilstoredectrstate', 'soilstoreevetrstate', 'soilstoregrassstate', 'soilstorebsoilstate']
['bldgsstate', 'pavedstate', 'dectrstate', 'evetrstate', 'grassstate', 'bsoilstate', 'waterstate']


In [18]:
df_var_nml_x.loc['albdectr0']
df_var_nml_x.index


Index(['dayssincerain', 'temp_c0', 'cbluse', 'snowuse', 'solweiguse',
       'netradiationmethod', 'emissionsmethod', 'storageheatmethod',
       'ohmincqf', 'stabilitymethod', 'roughlenheatmethod',
       'roughlenmommethod', 'smdmethod', 'waterusemethod', 'tstep',
       'resolutionfilesin', 'resolutionfilesinestm', 'resolutionfilesout',
       'leavesoutinitially', 'gdd_1_0', 'gdd_2_0', 'laiinitialevetr',
       'laiinitialdectr', 'laiinitialgrass', 'albevetr0', 'albdectr0',
       'albgrass0', 'decidcap0', 'porosity0', 'soilstorepavedstate',
       'soilstorebldgsstate', 'soilstoreevetrstate', 'soilstoredectrstate',
       'soilstoregrassstate', 'soilstorebsoilstate', 'snowinitially',
       'snowwaterpavedstate', 'snowwaterbldgsstate', 'snowwaterevetrstate',
       'snowwaterdectrstate', 'snowwatergrassstate', 'snowwaterbsoilstate',
       'snowwaterwaterstate', 'snowpackpaved', 'snowpackbldgs',
       'snowpackevetr', 'snowpackdectr', 'snowpackgrass', 'snowpackbsoil',
       'sno

#### generate `df_var_initcond`

In [19]:
dict_var_initcond_desc = {}
for var in dict_initcond_related:
    var_related = dict_initcond_related[var]
    desc = df_var_nml_x.loc[var_related, 'Description']
    desc = desc if isinstance(desc, str) else '\n'.join(desc.tolist())
    print(var_related)
    var_related = df_var_nml_x.loc[var_related, 'SUEWS-related variables']
    var_related = var_related if isinstance(
        var_related, str) else ', '.join(var_related.tolist())
    print(var_related)
    dict_var = {
        var: pd.Series({
            'Description': desc,
            'SUEWS-related variables': var_related,
        }),
    }

    dict_var_initcond_desc.update(dict_var)
df_initcond_desc = pd.concat(dict_var_initcond_desc).unstack()
df_initcond_dim = df_var_dim.loc[df_initcond_desc.index]
df_var_initcond = pd.concat([df_initcond_dim, df_initcond_desc], axis=1)
df_var_initcond



albdectr0
albDecTr0
albevetr0
albEveTr0
albgrass0
albGrass0
decidcap0
decidCap0
['laiinitialdectr', 'laiinitialevetr', 'laiinitialgrass']
LAIinitialDecTr, LAIinitialEveTr, LAIinitialGrass
porosity0
porosity0
snowalb0
SnowAlb0
['snowwaterbldgsstate', 'snowwaterpavedstate', 'snowwaterdectrstate', 'snowwaterevetrstate', 'snowwatergrassstate', 'snowwaterbsoilstate', 'snowwaterwaterstate']
SnowWaterBldgsState, SnowWaterPavedState, SnowWaterDecTrState, SnowWaterEveTrState, SnowWaterGrassState, SnowWaterBSoilState, SnowWaterWaterState
['snowdensbldgs', 'snowdenspaved', 'snowdensdectr', 'snowdensevetr', 'snowdensgrass', 'snowdensbsoil', 'snowdenswater']
SnowDensBldgs, SnowDensPaved, SnowDensDecTr, SnowDensEveTr, SnowDensGrass, SnowDensBSoil, SnowDensWater
['snowfracbldgs', 'snowfracpaved', 'snowfracdectr', 'snowfracevetr', 'snowfracgrass', 'snowfracbsoil', 'snowfracwater']
SnowFracBldgs, SnowFracPaved, SnowFracDecTr, SnowFracEveTr, SnowFracGrass, SnowFracBSoil, SnowFracWater
['snowpackbldgs', 

,Dimensionality,Description,SUEWS-related variables
albdectr_id,0,Albedo of deciduous surface `DecTr` on day 0 o...,albDecTr0
albevetr_id,0,Albedo of evergreen surface `EveTr` on day 0 o...,albEveTr0
albgrass_id,0,Albedo of grass surface `Grass` on day 0 of run,albGrass0
decidcap_id,0,Storage capacity of deciduous surface `DecTr` ...,decidCap0
lai_id,"(3,)",Initial LAI for deciduous trees `DecTr`.\nInit...,"LAIinitialDecTr, LAIinitialEveTr, LAIinitialGrass"
porosity_id,0,Porosity of deciduous vegetation on day 0 of run.,porosity0
snowalb,0,Initial snow albedo,SnowAlb0
snowdens,"(7,)",Initial snow density on buildings `Bldgs`\nIni...,"SnowDensBldgs, SnowDensPaved, SnowDensDecTr, S..."
snowfrac,"(7,)",Initial plan area fraction of snow on building...,"SnowFracBldgs, SnowFracPaved, SnowFracDecTr, S..."
snowpack,"(7,)",Initial snow water equivalent if the snow on b...,"SnowPackBldgs, SnowPackPaved, SnowPackDecTr, S..."


In [20]:
df_initcond_desc
# dict_initcond_related['albdectr_id']
df_var_nml_x.sort_index()


,Description,SUEWS-related variables
var,,
albdectr0,Albedo of deciduous surface `DecTr` on day 0 o...,albDecTr0
albevetr0,Albedo of evergreen surface `EveTr` on day 0 o...,albEveTr0
albgrass0,Albedo of grass surface `Grass` on day 0 of run,albGrass0
bldgsstate,Initial wetness condition on `Bldgs`,BldgsState
bsoilstate,Initial wetness condition on `BSoil`,BSoilState
cbluse,Determines whether a CBL slab model is used to...,CBLuse
dayssincerain,Days since rain [d],DaysSinceRain
decidcap0,Storage capacity of deciduous surface `DecTr` ...,decidCap0
dectrstate,Initial wetness condition on `DecTr`,DecTrState


## summarise the `DataFrame`s

### combine all `df_var_xx`s

In [21]:
df_var_supy_docs = pd.concat([df_var_site, df_var_runcontrol, df_var_initcond])
df_var_supy_docs.head()


,Dimensionality,Description,SUEWS-related variables
enddls,0,End of the day light savings [DOY],EndDLS
tau_a,0,Time constant for snow albedo aging in cold sn...,tau_a
ohm_threshsw,"(8,)",Temperature threshold determining whether summ...,OHMThresh_SW
snowdensmax,0,Maximum snow density [kg |m^-3|],SnowDensMax
raincover,0,Limit when surface totally covered with water ...,LUMPS_Cover


### these variables are for internal use and thus not released for users

In [22]:
set_input - set(df_var_supy_docs.index)


{'dqndt',
 'dqnsdt',
 'dt_since_start',
 'gdd_id',
 'gridiv',
 'hdd_id',
 'icefrac',
 'len_sim',
 'metforcingblock',
 'numcapita',
 'qn1_av',
 'qn1_s_av',
 'snowfallcum',
 'tair24hr',
 'tstep_prev',
 'wuday_id'}

### deal with dim info

In [23]:
df_var_supy_docs.loc[df_var_supy_docs.Dimensionality != 0]


,Dimensionality,Description,SUEWS-related variables
ohm_threshsw,"(8,)",Temperature threshold determining whether summ...,OHMThresh_SW
ie_m,"(3,)",Coefficient for manual irrigation model [mm |d...,"Ie_m1, Ie_m2, Ie_m3"
alpha_bioco2,"(3,)",The mean apparent ecosystem quantum. Represent...,alpha
min_res_bioco2,"(3,)",Minimum soil respiration rate (for cold-temper...,min_respi
sathydraulicconduct,"(7,)",Hydraulic conductivity for saturated soil [mm ...,SatHydraulicCond
soildepth,"(7,)",Depth of soil beneath the surface [mm],SoilDepth
basete,"(3,)",Base temperature for initiating sensesance deg...,BaseTe
snowd,"(7,)",Limit for the snow water equivalent when snow ...,SnowLimPatch
gddfull,"(3,)",The growing degree days (GDD) needed for full ...,GDDFull
trafficrate,"(2,)",Weekday traffic rate [veh km |m^-2| s-1] Can b...,"TrafficRate_WD, TrafficRate_WE"


#### 1D+ data

In [24]:
df_1Dplus = df_var_supy_docs.loc[df_var_supy_docs.Dimensionality != 0]
pos_1D = df_1Dplus.Dimensionality.map(len) == 1
df_1D = df_1Dplus.loc[pos_1D]
pos_23D = df_1Dplus.Dimensionality.map(len) > 1
df_23D = df_1Dplus.loc[pos_23D]



In [25]:
df_1D



,Dimensionality,Description,SUEWS-related variables
ohm_threshsw,"(8,)",Temperature threshold determining whether summ...,OHMThresh_SW
ie_m,"(3,)",Coefficient for manual irrigation model [mm |d...,"Ie_m1, Ie_m2, Ie_m3"
alpha_bioco2,"(3,)",The mean apparent ecosystem quantum. Represent...,alpha
min_res_bioco2,"(3,)",Minimum soil respiration rate (for cold-temper...,min_respi
sathydraulicconduct,"(7,)",Hydraulic conductivity for saturated soil [mm ...,SatHydraulicCond
soildepth,"(7,)",Depth of soil beneath the surface [mm],SoilDepth
basete,"(3,)",Base temperature for initiating sensesance deg...,BaseTe
snowd,"(7,)",Limit for the snow water equivalent when snow ...,SnowLimPatch
gddfull,"(3,)",The growing degree days (GDD) needed for full ...,GDDFull
trafficrate,"(2,)",Weekday traffic rate [veh km |m^-2| s-1] Can b...,"TrafficRate_WD, TrafficRate_WE"


In [26]:
df_23D



,Dimensionality,Description,SUEWS-related variables
humactivity_24hr,"(24, 2)",Code linking to `ActivityProfWD` in `SUEWS_Pro...,"ActivityProfWD, ActivityProfWE"
storedrainprm,"(6, 7)",Coefficient D0 [mm |h^-1|] used in :option:`Dr...,"DrainageCoef1, DrainageCoef2, DrainageEq, Stor..."
waterdist,"(8, 6)",Fraction of water going to ``BSoil``\nFraction...,"ToBSoil, ToBldgs, ToDecTr, ToEveTr, ToGrass, T..."
laipower,"(4, 3)",a parameter required by LAI calculation in `LA...,"LeafGrowthPower1, LeafGrowthPower2, LeafOffPow..."
wuprofm_24hr,"(24, 2)","Code for water use profile (manual irrigation,...","WaterUseProfManuWD, WaterUseProfManuWE"
ohm_coef,"(8, 4, 3)",Coefficient for Q* term [-]\nCoefficient for `...,"a1, a2, a3"
snowprof_24hr,"(24, 2)",Code for snow clearing profile (weekdays) link...,"SnowClearingProfWD, SnowClearingProfWE"
traffprof_24hr,"(24, 2)",Code for traffic activity profile (weekdays) l...,"TraffProfWD, TraffProfWE"
wuprofa_24hr,"(24, 2)",Code for water use profile (automatic irrigati...,"WaterUseProfAutoWD, WaterUseProfAutoWE"
ahprof_24hr,"(24, 2)",Code linking to `EnergyUseProfWD` in `SUEWS_Pr...,"EnergyUseProfWD, EnergyUseProfWE"


In [27]:
df_1D.Dimensionality.unique()



array([(8,), (3,), (7,), (2,)], dtype=object)

In [28]:
df_23D.Dimensionality.unique()



array([(24, 2), (6, 7), (8, 6), (4, 3), (8, 4, 3)], dtype=object)

In [29]:
dict_dim_anno = {
    0:
    'Scalar',
    (2, ):
    '2: Weekday and Weekend',
    (3, ):
    '3: See variable description for specifics',
    (7, ):
    '7: Seven SUEWS land cover types in the order of [paved; buildings; evergreen trees/shrubs; deciduous trees/shrubs; grass; bare soil and water]: https://suews-docs.readthedocs.io/en/latest/introduction.html',
    (8, ):
    '8: Seven SUEWS land cover types and one extra land cover type (currently NOT used)',
    (24, 2):
    '24: hours of a day; 2: Weekday and Weekend',
    (8, 4, 3):
    '; '.join([
        '8: Seven SUEWS land cover types and one extra land cover type (currently NOT used)',
        '4: SummerWet, SummerDry, WinterWet, WinterDry',
        '3: a1, a2, a3'
    ]),
    (4, 3):
    '4: See variable description for specifics; 3: Three vegetated land cover types (`EveTr`, `DecTr`, `Grass`)',
    (8, 6):
    '8: Seven SUEWS land cover types and Runoff/SoilStore as water receiver; 6: SUEWS land cover types other than water as water contributors',
    (6, 7):
    '6: See variable description for specifics; 7: Seven SUEWS land cover types in the order of [paved; buildings; evergreen trees/shrubs; deciduous trees/shrubs; grass; bare soil and water]: https://suews-docs.readthedocs.io/en/latest/introduction.html',
}



In [30]:
df_var_dim_anno = df_var_supy_docs['Dimensionality'].map(
    dict_dim_anno).rename('Dimensionality Remarks').to_frame()
# correct the `(7,)` for week days
anno_week = '7: Seven days of a week: from Sunday to Saturday'
df_var_dim_anno.loc[
    df_var_dim_anno.index.str.contains('daywat'),
    'Dimensionality Remarks'] = anno_week
df_var_dim_anno.sort_index().iloc[:36]


,Dimensionality Remarks
aerodynamicresistancemethod,Scalar
ah_min,2: Weekday and Weekend
ah_slope_cooling,2: Weekday and Weekend
ah_slope_heating,2: Weekday and Weekend
ahprof_24hr,24: hours of a day; 2: Weekday and Weekend
alb,7: Seven SUEWS land cover types in the order o...
albdectr_id,Scalar
albevetr_id,Scalar
albgrass_id,Scalar
albmax_dectr,Scalar


#### attach dim info to df_docs

In [31]:
df_var_supy_docs_dim = pd.concat([df_var_supy_docs, df_var_dim_anno], axis=1)
df_var_supy_docs_dim


,Dimensionality,Description,SUEWS-related variables,Dimensionality Remarks
enddls,0,End of the day light savings [DOY],EndDLS,Scalar
tau_a,0,Time constant for snow albedo aging in cold sn...,tau_a,Scalar
ohm_threshsw,"(8,)",Temperature threshold determining whether summ...,OHMThresh_SW,8: Seven SUEWS land cover types and one extra ...
snowdensmax,0,Maximum snow density [kg |m^-3|],SnowDensMax,Scalar
raincover,0,Limit when surface totally covered with water ...,LUMPS_Cover,Scalar
ie_m,"(3,)",Coefficient for manual irrigation model [mm |d...,"Ie_m1, Ie_m2, Ie_m3",3: See variable description for specifics
alpha_bioco2,"(3,)",The mean apparent ecosystem quantum. Represent...,alpha,3: See variable description for specifics
frfossilfuel_heat,0,Fraction of fossil fuels used for building hea...,FrFossilFuel_Heat,Scalar
zdm_in,0,Zero-plane displacement [m],zd,Scalar
crwmin,0,Minimum water holding capacity of snow [mm],CRWMin,Scalar


### add docs URL to SUEWS related variables

In [32]:
opt_str = 'SnowAlb0'
url_str = str(gen_url_option(opt_str.lower()))
f'`{opt_str} <{url_str}>`_'



'`SnowAlb0 <https://suews-docs.readthedocs.io/en/latest/input_files/Initial_Conditions/Snow_related_parameters.html#cmdoption-arg-snowalb0>`_'

In [33]:
def gen_rst_url_opt(opt_str):
    url_str = str(gen_url_option(opt_str.lower()))
    rst_str = f'`{opt_str} <{url_str}>`_'
    return rst_str


gen_rst_url_opt('QF0_BEU_WD')



'`QF0_BEU_WD <https://suews-docs.readthedocs.io/en/latest/input_files/SUEWS_SiteInfo/Input_Options.html#cmdoption-arg-qf0-beu-wd>`_'

In [34]:
'QF0_BEU_WD, QF0_BEU_WE'.split(',')



['QF0_BEU_WD', ' QF0_BEU_WE']

In [35]:
# split multiple opts into a list of them
# then generate the links
def gen_rst_url_split_opts(opts_str):
    if opts_str is not 'None':
        list_opts = opts_str.split(',')
        list_rst = [gen_rst_url_opt(opt.strip()) for opt in list_opts]
        list_url_rst = ', '.join(list_rst)
    else:
        list_url_rst = 'None'
    return list_url_rst


gen_rst_url_split_opts('None')



'None'

In [36]:
xx = df_var_supy_docs_dim.loc[:, 'SUEWS-related variables'].sort_index()
xx.loc[xx.isna()]



aerodynamicresistancemethod    NaN
diagnose                       NaN
diagqn                         NaN
diagqs                         NaN
evapmethod                     NaN
laicalcyes                     NaN
veg_type                       NaN
Name: SUEWS-related variables, dtype: object

In [37]:
df_var_supy_docs_dim_url = df_var_supy_docs_dim.copy().fillna('None')
df_var_supy_docs_dim_url['SUEWS-related variables'] = df_var_supy_docs_dim_url['SUEWS-related variables'].map(
    gen_rst_url_split_opts)
# df_var_supy_docs_dim_url=df_var_supy_docs_dim.transform({'SUEWS-related variables':gen_rst_url_split_opts})
df_var_supy_docs_dim_url


,Dimensionality,Description,SUEWS-related variables,Dimensionality Remarks
enddls,0,End of the day light savings [DOY],`EndDLS <https://suews-docs.readthedocs.io/en/...,Scalar
tau_a,0,Time constant for snow albedo aging in cold sn...,`tau_a <https://suews-docs.readthedocs.io/en/l...,Scalar
ohm_threshsw,"(8,)",Temperature threshold determining whether summ...,`OHMThresh_SW <https://suews-docs.readthedocs....,8: Seven SUEWS land cover types and one extra ...
snowdensmax,0,Maximum snow density [kg |m^-3|],`SnowDensMax <https://suews-docs.readthedocs.i...,Scalar
raincover,0,Limit when surface totally covered with water ...,`LUMPS_Cover <https://suews-docs.readthedocs.i...,Scalar
ie_m,"(3,)",Coefficient for manual irrigation model [mm |d...,`Ie_m1 <https://suews-docs.readthedocs.io/en/l...,3: See variable description for specifics
alpha_bioco2,"(3,)",The mean apparent ecosystem quantum. Represent...,`alpha <https://suews-docs.readthedocs.io/en/l...,3: See variable description for specifics
frfossilfuel_heat,0,Fraction of fossil fuels used for building hea...,`FrFossilFuel_Heat <https://suews-docs.readthe...,Scalar
zdm_in,0,Zero-plane displacement [m],`zd <https://suews-docs.readthedocs.io/en/late...,Scalar
crwmin,0,Minimum water holding capacity of snow [mm],`CRWMin <https://suews-docs.readthedocs.io/en/...,Scalar


### clean description

In [38]:
# drop duplicates
xx = df_var_supy_docs_dim_url.loc[:, 'Description']    .str.split('\n')    .map(
    lambda x: pd.Series(x).drop_duplicates().values)    .str.join(';;')
df_var_supy_docs_dim_url.loc[:, 'Description'] = xx
df_var_supy_docs_dim_url.head()



,Dimensionality,Description,SUEWS-related variables,Dimensionality Remarks
enddls,0,End of the day light savings [DOY],`EndDLS <https://suews-docs.readthedocs.io/en/...,Scalar
tau_a,0,Time constant for snow albedo aging in cold sn...,`tau_a <https://suews-docs.readthedocs.io/en/l...,Scalar
ohm_threshsw,"(8,)",Temperature threshold determining whether summ...,`OHMThresh_SW <https://suews-docs.readthedocs....,8: Seven SUEWS land cover types and one extra ...
snowdensmax,0,Maximum snow density [kg |m^-3|],`SnowDensMax <https://suews-docs.readthedocs.i...,Scalar
raincover,0,Limit when surface totally covered with water ...,`LUMPS_Cover <https://suews-docs.readthedocs.i...,Scalar


In [39]:
# set default values
df_var_supy_docs_dim_url = df_var_supy_docs_dim_url.replace(
    {'Description': {'None': 'Internal use. Please DO NOT modify'}})
df_var_supy_docs_dim_url.index = df_var_supy_docs_dim_url.index.rename(
    'variable')



In [40]:
# # save auto-generation-safe entries
# df_var_supy_auto = df_var_supy_docs_dim_url.copy()[~pos_entry_manual]
# df_var_supy_auto.to_csv('df_var_supy_auto.csv')


### manipulate some messy entries

In [41]:
# filter manual-work-demanding entries
pos_entry_manual = df_var_supy_docs_dim_url.loc[:, 'Description'].str.contains(
    ';;')
df_var_supy_manual = df_var_supy_docs_dim_url.copy()[pos_entry_manual]
# df_var_supy_manual.to_csv('df_var_supy_manual.csv')
df_var_supy_manual.index



Index(['ie_m', 'trafficrate', 'ie_a', 'daywatper', 'humactivity_24hr', 'qf_c',
       'storedrainprm', 'ah_min', 'waterdist', 'laipower', 'ah_slope_cooling',
       'wuprofm_24hr', 'ohm_coef', 'daywat', 't_critic_heating',
       'snowprof_24hr', 'qf_a', 'sfr', 'traffprof_24hr', 'qf_b',
       't_critic_cooling', 'wuprofa_24hr', 'ah_slope_heating', 'ahprof_24hr',
       'popprof_24hr', 'lai_id', 'snowdens', 'snowfrac', 'snowpack',
       'snowwater', 'soilstore_id', 'state_id'],
      dtype='object', name='variable')

In [42]:
# dict for manual edits of several entries
dict_var_desc_manual = {
    # surface fractions
    'sfr': 'Surface cover fractions.',
    # ohm coefficients:
    'ohm_coef': 'Coefficients for OHM calculation.',
    # profiles:
    'ahprof_24hr': 'Hourly profile values used in energy use calculation.',
    'traffprof_24hr': 'Hourly profile values used in traffic activity calculation.',
    'popprof_24hr': 'Hourly profile values used in dynamic population estimation.',
    'humactivity_24hr': 'Hourly profile values used in human activity calculation.',
    'wuprofa_24hr': 'Hourly profile values used in automatic irrigation.',
    'wuprofm_24hr': 'Hourly profile values used in manual irrigation.',
    'snowprof_24hr': 'Hourly profile values used in snow clearing.',
    # QF related:
    'ah_min': 'Minimum QF values.',
    'ah_slope_heating': 'Heating slope of QF calculation.',
    'ah_slope_cooling': 'Cooling slope of QF calculation.',
    'qf_a': 'Base value for QF calculation.',
    'qf_b': 'Parameter related to heating degree days.',
    'qf_c': 'Parameter related to heating degree days.',
    't_critic_heating': 'Critical heating temperature.',
    't_critic_cooling': 'Critical cooling temperature.',
    'trafficrate': 'Traffic rate used for CO2 flux calculation.',
    # irrigation related:
    'daywat': 'Irrigation flag: 1 for on and 0 for off.',
    'daywatper': 'Fraction of properties using irrigation for each day of a week.',
    'ie_a': 'Coefficient for automatic irrigation model.',
    'ie_m': 'Coefficient for manual irrigation model.',
    # water redistribution related:
    'storedrainprm': 'Coefficients used in drainage calculation.',
    'waterdist': 'Fraction of water redistribution',
    # snow related:
    'snowdens': 'Initial snow density of each land cover.',
    'snowfrac': 'Initial plan area fraction of snow on each land cover`',
    'snowpack': 'Initial snow water equivalent on each land cover',
    'snowwater': 'Initial amount of liquid water in the snow on each land cover',
    # LAI related:
    'laipower': 'parameters required by LAI calculation.',
    'lai_id': 'Initial LAI values.',
    # wetness related:
    'soilstore_id': 'Initial water stored in soil beneath each land cover',
    'state_id': 'Initial wetness condition on each land cover',
}
ser_desc_manual=pd.Series(dict_var_desc_manual).rename('Description')
# df_var_supy_manual.loc[:, 'Description'] = ser_desc_manual

In [43]:
# update those entries with manual definitions
df_var_supy_docs_mod = df_var_supy_docs_dim_url.copy()
df_var_supy_docs_mod['Description'].update(ser_desc_manual)
df_var_supy_docs_mod.loc[:,'Description'] 
# desc_mod


variable
enddls                                 End of the day light savings [DOY]
tau_a                   Time constant for snow albedo aging in cold sn...
ohm_threshsw            Temperature threshold determining whether summ...
snowdensmax                              Maximum snow density [kg |m^-3|]
raincover               Limit when surface totally covered with water ...
ie_m                             Coefficient for manual irrigation model.
alpha_bioco2            The mean apparent ecosystem quantum. Represent...
frfossilfuel_heat       Fraction of fossil fuels used for building hea...
zdm_in                                        Zero-plane displacement [m]
crwmin                        Minimum water holding capacity of snow [mm]
min_res_bioco2          Minimum soil respiration rate (for cold-temper...
fcef_v_kgkm                              CO2 emission factor [kg |km^-1|]
sathydraulicconduct     Hydraulic conductivity for saturated soil [mm ...
albmax_dectr            Effec

### save results

In [44]:
df_var_supy_docs_mod.to_csv('./df_var_supy.csv')

